In [9]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Hàm tải và chuẩn bị dữ liệu từ thư mục ảnh
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            label = filename.split("_")[0]
            if label in ["right", "forward", "left"]:
                img = cv2.imread(os.path.join(data_dir, filename))
                img = cv2.resize(img, (224, 224))  # Resize về kích thước 224x224
                images.append(img)
                labels.append(["right", "forward", "left"].index(label))
    return np.array(images), np.array(labels)

# Tải dữ liệu và chia tập huấn luyện/validation
data_dir = "dataset_clean"  # Thay bằng đường dẫn thực tế đến thư mục chứa ảnh
images, labels = load_data(data_dir)
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu (MobileNetV2 yêu cầu giá trị pixel trong [-1, 1])
train_images = tf.keras.applications.mobilenet_v2.preprocess_input(train_images)
val_images = tf.keras.applications.mobilenet_v2.preprocess_input(val_images)

# Định nghĩa mô hình với MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Không huấn luyện lại MobileNetV2

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation="softmax")  # 3 lớp đầu ra: right, forward, left
])

# Compile mô hình
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Huấn luyện mô hình
history = model.fit(train_images, train_labels, epochs=50, validation_data=(val_images, val_labels))

# Vẽ đồ thị loss
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("loss_plot_clearn.png")  # Lưu đồ thị thành file
plt.close()

# Dự đoán trên tập validation
val_predictions = model.predict(val_images)
val_predictions = np.argmax(val_predictions, axis=1)  # Chuyển từ xác suất sang nhãn

# Tính confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["right", "forward", "left"], 
            yticklabels=["right", "forward", "left"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix on Validation Set")
plt.savefig("confusion_matrix_clean.png")  # Lưu confusion matrix thành file
plt.close()

# Lưu mô hình dưới dạng .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("steering_model_clean.tflite", "wb") as f:
    f.write(tflite_model)


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 734ms/step - accuracy: 0.4762 - loss: 1.1476 - val_accuracy: 0.4545 - val_loss: 1.0496
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 676ms/step - accuracy: 0.5717 - loss: 0.9597 - val_accuracy: 0.5212 - val_loss: 0.9427
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 666ms/step - accuracy: 0.5996 - loss: 0.8834 - val_accuracy: 0.5333 - val_loss: 0.9019
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 659ms/step - accuracy: 0.6379 - loss: 0.8356 - val_accuracy: 0.5515 - val_loss: 0.9079
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 655ms/step - accuracy: 0.6038 - loss: 0.8443 - val_accuracy: 0.5333 - val_loss: 0.9053
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 649ms/step - accuracy: 0.6473 - loss: 0.7932 - val_accuracy: 0.5515 - val_loss: 0.8855
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 649ms/step - accuracy: 0.6841 - loss: 0.7586 - val_accuracy: 0.5636 - val_loss: 0.8845
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 664ms/step - accuracy: 0.6668 - loss: 0.7634 - val_accu

INFO:tensorflow:Assets written to: C:\Users\nguye\AppData\Local\Temp\tmpqftffywf\assets


Saved artifact at 'C:\Users\nguye\AppData\Local\Temp\tmpqftffywf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_470')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2984960804992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960810624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960812384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960809568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960811152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960818720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960813968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960814320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960815200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2984960817488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [10]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Dự đoán trên tập validation
val_predictions = model.predict(val_images)
val_predictions = np.argmax(val_predictions, axis=1)  # Chuyển từ xác suất sang nhãn

# Tính confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["right", "forward", "left"], 
            yticklabels=["right", "forward", "left"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix on Validation Set")
plt.savefig("confusion_matrix_clean.png")  # Lưu confusion matrix thành file
plt.close()

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 494ms/step


In [11]:

# # Tải mô hình .tflite
# interpreter = tf.lite.Interpreter(model_path="steering_model_clean.tflite")
# interpreter.allocate_tensors()

# # Lấy thông tin input và output của mô hình
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Kiểm tra trên luồng video
# stream_url = "http://192.168.46.130:81/stream"
# cap = cv2.VideoCapture(stream_url)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Không thể đọc luồng video.")
#         break

#     # Chuẩn bị frame cho dự đoán
#     frame_resized = cv2.resize(frame, (224, 224))
#     frame_processed = tf.keras.applications.mobilenet_v2.preprocess_input(frame_resized)
#     input_data = np.expand_dims(frame_processed, axis=0).astype(np.float32)

#     # Đặt dữ liệu đầu vào và chạy dự đoán
#     interpreter.set_tensor(input_details[0]["index"], input_data)
#     interpreter.invoke()

#     # Lấy kết quả dự đoán
#     output_data = interpreter.get_tensor(output_details[0]["index"])
#     prediction = np.argmax(output_data[0])
#     result = ["right", "forward", "left"][prediction]
#     print(f"Dự đoán: {result}")

#     # Hiển thị frame (tùy chọn)
#     cv2.imshow("Video Stream", frame)
#     if cv2.waitKey(1) & 0xFF == ord("q"):  # Nhấn 'q' để thoát
#         break

# # Giải phóng tài nguyên
# cap.release()
# cv2.destroyAllWindows()